In [ ]:
!pip install numpy scipy

In [ ]:
import numpy as np
from scipy.io.wavfile import write
import os

sample_rate = 44100
duration = 1.0  # seconds
base_freqs = [250, 500, 1000, 2000, 4000, 8000]
base_vols_db = [20, 40]

# Fix: use practical volume values directly
volume_map = {
    20: 0.2,
    40: 0.5
}

def generate_tone(freq, volume, duration, sample_rate):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    waveform = np.sin(2 * np.pi * freq * t) * volume
    return np.int16(waveform * 32767)

def generate_stereo_tone(freq, vol, ear):
    mono = generate_tone(freq, vol, duration, sample_rate)
    if ear == 'left':
        stereo = np.column_stack((mono, np.zeros_like(mono)))
    elif ear == 'right':
        stereo = np.column_stack((np.zeros_like(mono), mono))
    else:
        stereo = np.column_stack((mono, mono))
    return stereo

# Output folder
output_dir = "audio"
os.makedirs(output_dir, exist_ok=True)

for freq in base_freqs:
    for db in base_vols_db:
        vol = volume_map[db]
        for ear in ['both', 'left', 'right']:
            stereo_audio = generate_stereo_tone(freq, vol, ear)
            filename = f"{freq}Hz_{db}dB" + (f"_{ear}" if ear != 'both' else "") + ".wav"
            filepath = os.path.join(output_dir, filename)
            write(filepath, sample_rate, stereo_audio)
            print(f"✅ Saved {filename}")

In [ ]:
import shutil
shutil.make_archive("hearing_test_audio", 'zip', output_dir)